In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.
tensorflow-metadata 1.13.1 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible.

# Lab 5: CSP and OR-Tools

In this lab, you will gain hands-on experience of using the CSP solver from the OR-Tools suite.
"OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming" ([source](https://developers.google.com/optimization)).

OR-Tools is being developed by Google, and it has been consistently winning competitions such as [MiniZinc](https://www.minizinc.org/challenge.html).  The suite includes several solvers for a range of mathematical programs including CP-SAT, which is a CSP solver based on a SAT solver.

## Installation of the Python package

Run the command at the top of the notebook to install OR-Tools.  It may take 10&ndash;20 seconds.

## Exercises
### Exercise 1

Study the example below.  It uses OR-Tools to solve problem (1)&ndash;(7):

$$
\begin{split}
& b \rightarrow (x > y), \qquad & (1)\\
& x \neq y,\ x \neq z,\ y \neq z, \qquad & (2)\\
& a \lor b, \qquad & (3)\\
& a = 0, \qquad & (4)\\
& 0 \le x, y, z \le 2, \qquad & (5)\\
& x, y, z \text{ are integers}, \qquad & (6)\\
& a, b \in \{ 0, 1 \}. \qquad & (7)\\
\end{split}
$$





In [2]:
from ortools.sat.python import cp_model # CP-SAT solver

# Create the 'model', i.e. the knowledge base:
model = cp_model.CpModel()

# Create the variables (this is the CSP version of FOL constants):
x = model.NewIntVar(0, 2, 'x')
y = model.NewIntVar(0, 2, 'y')
z = model.NewIntVar(0, 2, 'z')

a = model.NewBoolVar('a')
b = model.NewBoolVar('b')

# Define the constraints:
model.Add(x > y).OnlyEnforceIf(b)
model.AddAllDifferent([x, y, z])
model.AddBoolOr([a, b]) # (a or b) has to hold
model.Add(a == 0)

# Create a solver and solve the model:
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Check if a solution was found:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # Print the solution
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
    print(f'a = {bool(solver.Value(a))}')
    print(f'b = {bool(solver.Value(b))}')
else:
    print('unsat')

x = 2
y = 0
z = 1
a = False
b = True


### Exercise 2

Come up with some modification of the formulation (1)&ndash;(8) that would make it unsatisfiable.  Copy the above code and made the change; verify using OR-Tools that the problem is indeed unsatisfiable.

In [10]:
# Your solution to exercise 2
from ortools.sat.python import cp_model # CP-SAT solver

# Create the 'model', i.e. the knowledge base:
model = cp_model.CpModel()

# Create the variables (this is the CSP version of FOL constants):
# x = model.NewIntVar(0, 2, 'x')
# y = model.NewIntVar(0, 2, 'y')
# z = model.NewIntVar(0, 2, 'z')
x = model.NewIntVar(0, 1, 'x')
y = model.NewIntVar(0, 1, 'y')
z = model.NewIntVar(0, 1, 'z')

a = model.NewBoolVar('a')
b = model.NewBoolVar('b')

# Define the constraints:
model.Add(x > y).OnlyEnforceIf(b)
model.AddAllDifferent([x, y, z])
model.AddBoolOr([a, b]) # (a or b) has to hold
model.Add(a == 0)

# Create a solver and solve the model:
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Check if a solution was found:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    # Print the solution
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
    print(f'a = {bool(solver.Value(a))}')
    print(f'b = {bool(solver.Value(b))}')
else:
    print('unsat')

unsat


### Exercise 3

Implement a solver of the Latin Square Problem using OR-Tools.  Use the approach with individual integer constants for each matrix element and explicit enumeration of constraints $m_{i,j} \neq m_{i,k}$, etc. (formulation (5)&ndash;(8) from Lab&nbsp;5).  Here is the CSP version of it (it is very similar to the FOL version):
$$
\begin{split}
& m_{i,j} \neq m_{k,j} \qquad & \forall i < k \in N,\ \forall j \in N, \qquad & (8)\\
& m_{i,j} \neq m_{i,k} & \forall i \in N,\ \forall j < k \in N, \qquad & (9)\\
& m_{i,j} = t & \forall (i, j, t) \in V, \qquad & (10)\\
& m_{i,j} \in N & \forall i, j \in N, \qquad & (11)
\end{split}
$$
where $N = \{ 1, 2, \ldots, n \}$.

In [11]:
# Your solution to exercise 3

from ortools.sat.python import cp_model

def solve_latin_square(n, V):

    model = cp_model.CpModel()

    # Your code here
    m = [[model.NewIntVar(1, n, f'cell_{i}_{j}') for j in range(n)] for i in range(n)]

    # Row constraints: all values in a row must be different
    for j in range(n):
        for i in range(n):
            for k in range(i + 1, n):
                model.Add(m[i][j] != m[k][j])

    # Column constraints: all values in a column must be different
    for i in range(n):
        for j in range(n):
            for k in range(j + 1, n):
                model.Add(m[i][j] != m[i][k])

    # Predefined values constraints
    for (x, y, t) in V:
        model.Add(m[x - 1][y - 1] == t)

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Check if a solution was found:
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for i in range(n):
            print(' '.join(str(solver.Value(m[i][j])) for j in range(n)))
    else:
        print('unsat')

solve_latin_square(10, [(1, 1, 2), (2, 2, 3)])

2 8 7 6 5 4 3 9 1 10
7 3 6 5 4 10 1 2 9 8
4 6 5 3 10 1 2 7 8 9
6 5 4 9 3 2 8 1 10 7
8 7 1 4 2 3 9 10 6 5
5 4 3 2 1 9 10 8 7 6
9 2 10 1 8 7 6 5 4 3
3 1 2 10 9 8 7 6 5 4
1 10 9 8 7 6 5 4 3 2
10 9 8 7 6 5 4 3 2 1


### Exercise 4

Compare the performances of your solvers based on Z3 and OR-Tools.  Which one is faster?

Your answer: OR-Tools

### Exercise 5

CSP has a standard constraint _AllDiff_ which requests that all the variables in a given set are assigned distinct values.
This constraint is particularly convenient in the formulation of the Latin square problem.
Mathematically, we can formulate the Latin square problem as follows.

Let $n$ be the size of the Latin square.
Let $N = \{ 1, 2, \ldots, n \}$ &ndash; we introduce this set for convenience.
Let $V$ be a set of tuples $(i, j, t)$ of known Latin square values, where $i$ and $j$ are the known element coordinates and $t$ is the known element value.
(For the example given in Lab 4, $V = \big\{ (1, 3, 1),\ (2, 1, 2),\ (3, 3, 3) \big\}$.)

$$
\begin{align}
& \text{AllDiff}(\{ m_{i,j} : i \in N \}) && \forall j \in N, && (12)\\
& \text{AllDiff}(\{ m_{i,j} : j \in N \}) && \forall i \in N, && (13)\\
& m_{i,j} = t && \forall (i, j, t) \in V, && (14)\\
& m_{i,j} \in N && \forall i, j \in N. && (15)
\end{align}
$$

Constraints (12) request that the values are not repeated within each column, constraints (13) request that the values are not repeated within each row and constraints (14) request that the known values are respected.
(15) define variables $m_{i,j}$ for $i, j \in N$ and their domains.

In [19]:
# Your solution to exercise 5

from ortools.sat.python import cp_model

def solve_latin_square_alldiff(n, V):

    model = cp_model.CpModel()

    # Your code here
    m = [[model.NewIntVar(1, n, f'cell_{i}_{j}') for j in range(n)] for i in range(n)]

    # Row constraints: all values in a row must be different
    for j in range(n):
        model.AddAllDifferent([m[i][j] for i in range(n)])

    # Column constraints: all values in a column must be different
    for i in range(n):
        model.AddAllDifferent([m[i][j] for j in range(n)])

    # Predefined values constraints
    for (x, y, t) in V:
        model.Add(m[x - 1][y - 1] == t)

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Check if a solution was found:
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for i in range(n):
            print(' '.join(str(solver.Value(m[i][j])) for j in range(n)))
    else:
        print('unsat')

solve_latin_square_alldiff(10, [(1, 1, 2), (2, 2, 3)])

2 4 5 10 6 7 8 3 9 1
8 3 10 5 7 1 6 9 4 2
10 1 4 2 8 9 5 6 7 3
9 8 6 3 1 10 4 7 2 5
7 5 2 8 9 3 1 4 6 10
6 9 7 1 2 5 10 8 3 4
5 2 1 7 4 6 3 10 8 9
4 10 9 6 3 2 7 1 5 8
3 6 8 9 10 4 2 5 1 7
1 7 3 4 5 8 9 2 10 6


### Exercise 6

Let us introduce a modification of the Latin Square Problem which we will call *Bounded Latin Square Problem*.
The Bounded Latin Square Problem is exactly the same as the Latin Square Problem but it adds one more condition:
$$
\max_{i, j \in \{ 1, \ldots, k \}} m_{i,j} < \min_{i, j \in \{ n - k + 1, \ldots, n \}} m_{i,j}, \qquad (16)
$$
where $k$ is a parameter, $1 \le k < n$.
In other words, the maximum element in the top left submatrix of size $k \times k$ is smaller than the minimum element in the bottom right submatrix of size $k \times k$.
See an example of a solution to the Bounded Latin Square below:

![Image](https://i.ibb.co/zNC88Q7/bounded-latin-square.png)

Here $n = 6$, $k = 2$ and $V = \emptyset$.

Implement a solver for the Bounded Latin Square Problem.
Use the `AddMaxEquality` and `AddMinEquality` functions (see the [documentation](https://developers.google.com/optimization/reference/python/sat/python/cp_model#addmaxequality)).
You may find that the documentation is scarce, but this is common in such specialist libraries, hence it is a useful exercise.

To use these functions, you will need _auxiliary variables_.
For example, constraint `AddMaxEquality` requests that variable `target` is equal to the maximum among the variables `variables`.
While it is clear which variables should be included in the list `variables`, what should be used for `target`?
This has to be a new (auxiliary) variable because we could not use any of the existing variables.

Write down (perhaps, on paper) your formulation of the Bounded Latin Square Problem and implement it in Python/OR Tools.

Ask for help if you get stuck.

In [20]:
# Your solution to exercise 6

from ortools.sat.python import cp_model

def solve_bounded_latin_square(n, k, V):

    model = cp_model.CpModel()

    m = [[model.NewIntVar(1, n, f'cell_{i}_{j}') for j in range(n)] for i in range(n)]

    for i in range(n):
        model.AddAllDifferent([m[i][j] for j in range(n)])

    for j in range(n):
        model.AddAllDifferent([m[i][j] for i in range(n)])

    for (x, y, t) in V:
        model.Add(m[x - 1][y - 1] == t)

    # Top-left submatrix of size k x k: find the maximum value
    top_left = [m[i][j] for i in range(k) for j in range(k)]
    max_top_left = model.NewIntVar(1, n, 'max_top_left')
    model.AddMaxEquality(max_top_left, top_left)

    # Bottom-right submatrix of size k x k: find the minimum value
    bottom_right = [m[i][j] for i in range(n - k, n) for j in range(n - k, n)]
    min_bottom_right = model.NewIntVar(1, n, 'min_bottom_right')
    model.AddMinEquality(min_bottom_right, bottom_right)

    # Constraint: max in top-left < min in bottom-right
    model.Add(max_top_left < min_bottom_right)

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Print the solution if found
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        for i in range(n):
            print(' '.join(str(solver.Value(m[i][j])) for j in range(n)))
    else:
        print("No solution exists")

solve_bounded_latin_square(6, 2, [(1, 1, 2), (2, 2, 3)])


2 1 5 6 4 3
1 3 6 5 2 4
6 5 2 4 3 1
5 6 4 3 1 2
4 2 3 1 6 5
3 4 1 2 5 6


### Exercise 7

It is actually very easy to get rid of the `AddMaxEquality` and `AddMinEquality` functions.
Modify your solver so that it does not use those functions.
Update the formulation accordingly.

Can you think of a way to formulate the Bounded Latin Square Problem with just one auxiliary variable?

In fact, there is a way to formulate it without auxiliary variables, but it is not particularly compact.
As an optional exercise, you may want to implement such a formulation and compare its performance to the performance of the formulation with auxiliary variables.

In [21]:
# Your solution to exercise 7
from ortools.sat.python import cp_model

def solve_bounded_latin_square(n, k, V):

    model = cp_model.CpModel()

    # Create a matrix of integer variables for the Latin square
    m = [[model.NewIntVar(1, n, f'cell_{i}_{j}') for j in range(n)] for i in range(n)]

    # Row constraints: all values in a row must be different
    for i in range(n):
        model.AddAllDifferent([m[i][j] for j in range(n)])

    # Column constraints: all values in a column must be different
    for j in range(n):
        model.AddAllDifferent([m[i][j] for i in range(n)])

    # Predefined values constraints
    for (x, y, t) in V:
        model.Add(m[x - 1][y - 1] == t)

    # Directly enforce that every element in the top-left kxk submatrix
    # is strictly less than every element in the bottom-right kxk submatrix
    for i1 in range(k):
        for j1 in range(k):
            for i2 in range(n - k, n):
                for j2 in range(n - k, n):
                    model.Add(m[i1][j1] < m[i2][j2])

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Print the solution if found
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        for i in range(n):
            print(' '.join(str(solver.Value(m[i][j])) for j in range(n)))
    else:
        print("No solution exists")

# Example usage
solve_bounded_latin_square(6, 2, [(1, 1, 2), (2, 2, 3)])



2 1 5 6 4 3
1 3 6 4 2 5
5 6 4 2 3 1
6 4 3 5 1 2
4 2 1 3 5 6
3 5 2 1 6 4


### Exercise 8

Let us introduce one more modification of the problem.
We will call it Conditional Bounded Latin Square Problem.
I will not explain it in any way other than the mathematical formulation:
$$
\begin{split}
& \begin{array}
\text{\text{AllDiff}}(\{ m_{i,j} : i \in N \}) & \forall j \in N, \qquad \qquad \qquad \qquad \qquad \quad & (17)\\
\text{AllDiff}(\{ m_{i,j} : j \in N \}) & \forall i \in N, & (18) \\
m_{i,j} = v & \forall (i, j, v) \in V, & (19) \\
m_{i,j} \in N & \forall i, j \in N, & (20) \\
\end{array}\\
& \displaystyle{\left(\max_{i, j \in \{ 1, \ldots, m \}} m_{i,j} \le k \right)~~ \text{or} ~~\left( \min_{i, j \in \{ n - m + 1, \ldots, n \}} m_{i,j} > n - k \right). \quad (21)}
\end{split}
$$

To implement this, you might want to use the [`OnlyEnforceIf`](https://developers.google.com/optimization/reference/python/sat/python/cp_model#onlyenforceif) function of the `Constraint` class.

In [ ]:
from ortools.sat.python import cp_model

def solve_conditional_bounded_latin_square(n, k, V):

    model = cp_model.CpModel()

    # Create a matrix of integer variables for the Latin square
    m_matrix = [[model.NewIntVar(1, n, f'cell_{i}_{j}') for j in range(n)] for i in range(n)]

    # Row constraints: all values in a row must be different
    for i in range(n):
        model.AddAllDifferent([m_matrix[i][j] for j in range(n)])

    # Column constraints: all values in a column must be different
    for j in range(n):
        model.AddAllDifferent([m_matrix[i][j] for i in range(n)])

    # Predefined values constraints
    for (x, y, v) in V:
        model.Add(m_matrix[x - 1][y - 1] == v)

    # Top-left submatrix of size k x k
    top_left = [m_matrix[i][j] for i in range(k) for j in range(k)]

    # Bottom-right submatrix of size k x k
    bottom_right = [m_matrix[i][j] for i in range(n - k, n) for j in range(n - k, n)]

    # Conditional constraints using OnlyEnforceIf
    max_top_left = model.NewIntVar(1, n, 'max_top_left')
    min_bottom_right = model.NewIntVar(1, n, 'min_bottom_right')
    condition1 = model.NewBoolVar('condition1')
    condition2 = model.NewBoolVar('condition2')

    model.AddMaxEquality(max_top_left, top_left)
    model.AddMinEquality(min_bottom_right, bottom_right)

    # max_top_left <= k if condition1 is true
    model.Add(max_top_left <= k).OnlyEnforceIf(condition1)

    # min_bottom_right > n - k if condition2 is true
    model.Add(min_bottom_right > n - k).OnlyEnforceIf(condition2)

    # Ensure at least one of the conditions is true
    model.AddBoolOr([condition1, condition2])

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Print the solution if found
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        for i in range(n):
            print(' '.join(str(solver.Value(m_matrix[i][j])) for j in range(n)))
    else:
        print("No solution exists")

# Example usage
solve_conditional_bounded_latin_square(10, 2, [(1, 1, 2), (2, 2, 3)])

2 10 3 4 5 7 8 9 1 6
4 3 2 6 9 1 10 5 8 7
10 4 5 8 7 9 2 6 3 1
9 5 7 10 3 6 1 2 4 8
8 9 10 1 4 5 6 7 2 3
5 1 9 2 10 3 7 8 6 4
7 8 6 3 1 10 9 4 5 2
1 6 4 9 2 8 3 10 7 5
6 2 1 7 8 4 5 3 9 10
3 7 8 5 6 2 4 1 10 9


**Warning.** The `OnlyEnforceIf` function is only implemented for linear constraints (the `Add` function), `AddBoolOr` and `AddBoolAnd`.  If you use it on a constraint of a different type such as `AddAllDifferent` then it will be silently ignored by OR-Tools.  (Questionable design choice by Google :) )